Before running the next cell please add a shortcut to the shared folder at the root of your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Learning_with_Density_Matrices_and_Random_Features-Supplementary_Material/qmc/tf')
import layers as layers
import models as models

/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.CrossProduct.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))
/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.DensityMatrix2Dist.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))
/usr/local/lib/python3.6/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking layers.DensityMatrixRegression.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


In [ ]:
import warnings
import numpy as np
from sklearn.kernel_approximation import RBFSampler

In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import gzip

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pickle

In [ ]:
import sklearn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer

In [ ]:
# from functions.keras_wrapper_qmc import KerasClassifier

# from sklearn.model_selection import RandomizedSearchCV, KFold
# from sklearn.metrics import make_scorer

In [ ]:
class LenetQMKDClassifierSGD(tf.keras.Model):
    """
    A Quantum Measurement Kernel Density Classifier model trainable using
    gradient descent.

    Arguments:
        input_dim: dimension of the input
        dim_x: dimension of the input quantum feature map
        num_classes: number of classes
        num_eig: Number of eigenvectors used to represent the density matrix.
                 a value of 0 or less implies num_eig = dim_x
        gamma: float. Gamma parameter of the RBF kernel to be approximated
        random_state: random number generator seed
    """
    def __init__(self, input_dim, dim_x, num_classes,  num_eig=0, gamma=1, random_state=None, fm_x=None):
        super(LenetQMKDClassifierSGD, self).__init__()

        self.conv_2d_1 = tf.keras.layers.Conv2D(filters=20, kernel_size=(5, 5), padding="same", activation='relu', input_shape=(input_dim, input_dim, 1))
        self.average_pooling_2d_1 = tf.keras.layers.AveragePooling2D()

        self.conv_2d_2 = tf.keras.layers.Conv2D(filters=50, kernel_size=(5, 5), padding="same", activation='relu')
        self.average_pooling_2d_2 = tf.keras.layers.AveragePooling2D()

        self.flatten = tf.keras.layers.Flatten()

        # self.dense_1 = tf.keras.layers.Dense(units=500, activation='relu')
        self.dense_1 = tf.keras.layers.Dense(units=84, activation='relu')

        # self.dense_2 = tf.keras.layers.Dense(units=84, activation='relu')

        if fm_x is None:
            self.fm_x = layers.QFeatureMapORF(
                input_dim=84,
                dim=dim_x, gamma=gamma, random_state=random_state)
        else:
            self.fm_x = fm_x
        self.dim_x = dim_x
        self.num_classes = num_classes
        self.qmd = []
        for _ in range(num_classes):
            self.qmd.append(layers.QMeasureDensityEig(dim_x, num_eig))
        self.gamma = gamma
        self.random_state = random_state

        
    def call(self, inputs):
        model = self.conv_2d_1(inputs)
        model = self.average_pooling_2d_1(model)
        model = self.conv_2d_2(model)
        model = self.average_pooling_2d_2(model)
        model = self.flatten(model)
        model = self.dense_1(model)
        # model = self.dense_2(model)


        psi_x = self.fm_x(model)
        probs = []
        for i in range(self.num_classes):
            probs.append(self.qmd[i](psi_x))
        posteriors = tf.stack(probs, axis=-1)
        posteriors = (posteriors /
                      tf.expand_dims(tf.reduce_sum(posteriors, axis=-1), axis=-1))
        return posteriors

    def set_rhos(self, rhos):
        for i in range(self.num_classes):
            self.qmd[i].set_rho(rhos[i])
        return


    def get_config(self):
        config = {
            "dim_x": self.dim_x,
            "num_classes": self.num_classes,
            "num_eig": self.num_eig,
            "gamma": self.gamma,
            "random_state": self.random_state
        }
        base_config = super().get_config()
        return {**base_config, **config}


In [ ]:
class Lenet(tf.keras.Model):
    """
    A Lenet model
    """
    def __init__(self):
        super(Lenet, self).__init__()

        self.conv_2d_1 = tf.keras.layers.Conv2D(filters=20, kernel_size=(5, 5), padding="same", activation='relu', input_shape=(input_dim, input_dim, 1))
        self.average_pooling_2d_1 = tf.keras.layers.AveragePooling2D()

        self.conv_2d_2 = tf.keras.layers.Conv2D(filters=50, kernel_size=(5, 5), padding="same", activation='relu')
        self.average_pooling_2d_2 = tf.keras.layers.AveragePooling2D()

        self.flatten = tf.keras.layers.Flatten()

        # self.dense_1 = tf.keras.layers.Dense(units=500, activation='relu')
        self.dense_1 = tf.keras.layers.Dense(units=84, activation='relu')

        # self.dense_2 = tf.keras.layers.Dense(units=84, activation='relu')

        self.softmax = tf.keras.layers.Dense(units=10, activation = 'softmax')

        
    def call(self, inputs):
        model = self.conv_2d_1(inputs)
        model = self.average_pooling_2d_1(model)
        model = self.conv_2d_2(model)
        model = self.average_pooling_2d_2(model)
        model = self.flatten(model)
        model = self.dense_1(model)
        # model = self.dense_2(model)

        return self.softmax(model)

### example

In [ ]:
!gdown --id 1q1bC5xP5SDYkggj6M5tclu3P0yI3YiTW


from load_cifar_10 import cifar10

train_images, train_labels, test_images, test_labels = cifar10(path='data', is_one_hot=False)

print(train_images.shape)
print(np.unique(train_labels))

#%%

train_images.shape

#%%

random_train = np.random.choice(range(train_images.shape[0]), train_images.shape[0], replace=False)
random_test = np.random.choice(range(test_images.shape[0]), test_images.shape[0], replace=False)


X_train = train_images[random_train,:]
y_train = train_labels[random_train]
X_test = test_images[random_test,:]
y_test = test_labels[random_test]
#%%


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)

Downloading...
From: https://drive.google.com/uc?id=1q1bC5xP5SDYkggj6M5tclu3P0yI3YiTW
To: /content/load_cifar_10.py
100% 3.88k/3.88k [00:00<00:00, 12.5MB/s]
(50000, 3072)
[0 1 2 3 4 5 6 7 8 9]
shape X_train :  (50000, 3072)
shape y_train :  (50000,)
shape X_test :  (10000, 3072)
shape y_test :  (10000,)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.9, random_state=42, stratify=y_train)
# X_test, _, y_test, _ = train_test_split(X_test, y_test, test_size=0.10, random_state=42, stratify=y_test)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
gammas = [2**i for i in range(-10,10)]

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [ ]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=192)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)

In [ ]:
input_dim = X_train.shape[1]
component_dim = 1000
num_classes = np.unique(y_train).shape[0]
gamma = 2**-5
random_state=0
num_eig=0
batch_size=8
epochs = 10

In [ ]:
def create_model(input_dim, num_classes, component_dim=100, gamma=1, lr=0.01, decay=0.,
                  random_state=None, eig_percentage=0, initialize_with_rff=False,
                  type_of_rff="rff", fix_rff=False):
    '''This is a model generating function so that we can search over neural net
    parameters and architecture'''

    num_eig = round(eig_percentage * component_dim)

    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, decay=decay)

    if type_of_rff == 'rff':
        fm_x = layers.QFeatureMapRFF(84, dim=component_dim, gamma=gamma, random_state=random_state)
    #else:
    #    fm_x = layers.QFeatureMapORF(84, dim=component_dim, gamma=gamma, random_state=random_state)

    if initialize_with_rff:
        qmkdc = models.QMKDClassifier(fm_x=fm_x, dim_x=component_dim, num_classes=num_classes)
        qmkdc.compile()
        qmkdc.fit(X_train, y_train, epochs=1, batch_size=batch_size, verbose=0)

    qmkdc1 = LenetQMKDClassifierSGD(input_dim=84, dim_x=component_dim, num_eig=num_eig,
                                      num_classes=num_classes, gamma=gamma, random_state=random_state, fm_x=fm_x)

    if fix_rff:
        qmkdc1.layers[0].trainable = False

    qmkdc1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

    if initialize_with_rff:
        qmkdc1.set_rhos(qmkdc.get_rhos())

    # qmkdc1.fit(X_train, y_train_bin, epochs=epochs, batch_size=batch_size)

    return qmkdc1

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/gdrive/My Drive/Universidad/doctorado_programacion/logs/cifar_lenet_qmkdc"

In [ ]:
# Creation of the model without random search. It can be used to verify the initial performance of the model.
import datetime, os
logdir = os.path.join("/gdrive/My Drive/Universidad/doctorado_programacion/logs/cifar_lenet_qmkdc", "cifar-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model_mnist = create_model(input_dim, num_classes, component_dim=1000, gamma=2**-6, lr=1e-05, decay=1e-9,
                  random_state=None, eig_percentage=0.1, initialize_with_rff=False,
                  type_of_rff="rff", fix_rff=False)

y_train_bin = tf.reshape(tf.keras.backend.one_hot(y_train, num_classes), (-1, num_classes))
y_test_bin = tf.reshape(tf.keras.backend.one_hot(y_test, num_classes), (-1, num_classes))
model_mnist.fit(X_train.reshape(-1,32,32,3), y_train_bin.numpy(), epochs=400, batch_size=batch_size, validation_data=(X_test.reshape(-1,32,32,3), y_test_bin.numpy()), 
            callbacks=[tensorboard_callback])

Epoch 1/400
  1/625 [..............................] - ETA: 0s - loss: 2.3933 - accuracy: 0.0000e+00WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
625/625 [==============================] - 24s 38ms/step - loss: 2.2915 - accuracy: 0.1238 - val_loss: 2.2588 - val_accuracy: 0.1652
Epoch 2/400
625/625 [==============================] - 23s 37ms/step - loss: 2.1795 - accuracy: 0.2056 - val_loss: 2.1000 - val_accuracy: 0.2402
Epoch 3/400
625/625 [==============================] - 23s 36ms/step - loss: 2.0575 - accuracy: 0.2558 - val_loss: 2.0287 - val_accuracy: 0.2679
Epoch 4/400
625/625 [==============================] - 23s 36ms/step - loss: 1.9972 - accuracy: 0.2824 - val_loss: 1.9718 - val_accuracy: 0.2888
Epoch 5/400
625/625 [==============================] -

In [ ]:
y_test_bin = tf.reshape(tf.keras.backend.one_hot(y_test, num_classes), (-1, num_classes))
out = model_mnist.predict(X_test)
accuracy_score(y_test, out.argmax(axis=1))

0.9051666666666667

In [ ]:
# model = Lenet()
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999), metrics=['accuracy'])
model.fit(X_train.reshape(-1,32,32,3), y_train_bin.numpy(), epochs=200, batch_size=32, validation_data=(X_test.reshape(-1,32,32,3), y_test_bin.numpy()), 
            callbacks=[tensorboard_callback])

Epoch 1/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1443 - accuracy: 0.6022 - val_loss: 1.2050 - val_accuracy: 0.5766
Epoch 2/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1436 - accuracy: 0.6033 - val_loss: 1.1958 - val_accuracy: 0.5774
Epoch 3/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1428 - accuracy: 0.6041 - val_loss: 1.1925 - val_accuracy: 0.5794
Epoch 4/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1411 - accuracy: 0.6042 - val_loss: 1.1850 - val_accuracy: 0.5840
Epoch 5/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1398 - accuracy: 0.6045 - val_loss: 1.1941 - val_accuracy: 0.5809
Epoch 6/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1390 - accuracy: 0.6055 - val_loss: 1.2039 - val_accuracy: 0.5774
Epoch 7/200
1563/1563 [==============================] - 6s 4ms/step - loss: 1.1381 - accuracy: 0.6040 - val_loss: 1.1862 - val_ac

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=16, verbose=1)

In [ ]:
num_classes

26

In [ ]:
2**30

1073741824

In [ ]:
# component dimension
from scipy.stats import randint
#components_dimensions = randint(20,1500)
components_dimensions = [1000]

# gamma
#gammas = [2 ** i for i in range(-25, 25)]
gammas = [2 ** i for i in range(4, 6)]

# number of eigen values
eig_values = [1 / 10, 25 / 100, 50 / 100, 1]

# initialize with rff
#initialize_with_rff = [True, False]
initialize_with_rff = [True]

# type_of_rff_components
type_of_rff = ['rff', 'orf']

# fix_rff
fix_rff = [True]

# learning algorithm parameters
lr = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
decay = [1e-6, 1e-9, 0]

# dictionary summary
param_grid = dict(input_dim=[input_dim], num_classes=[num_classes], component_dim=components_dimensions, gamma=gammas,
                  lr=lr, decay=decay,
                  random_state=[None], eig_percentage=eig_values, initialize_with_rff=initialize_with_rff,
                  type_of_rff=type_of_rff, fix_rff=fix_rff
                  )

In [ ]:
grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid,
                              verbose=20, n_iter=1, n_jobs=1)

In [ ]:
# grid_result = grid.fit(X_train, y_train)
y_train_bin = tf.reshape(tf.keras.backend.one_hot(y_train, num_classes), (-1, num_classes)).numpy()

In [ ]:
cv_results_df = None

for i in range(25):
  grid_result = grid.fit(X_train, y_train_bin)

  if cv_results_df is None:
    cv_results_df = pd.DataFrame(grid_result.cv_results_)
  else:
    cv_results_df = pd.concat([cv_results_df, pd.DataFrame(grid_result.cv_results_)])

  cv_results_df.to_csv('qmkdc cifar random search.csv')